In [1]:
## need to execute if use sagemaker prebuilt docker image as base
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [2]:
repo_name = "sagemaker-torch113-cu117-jre" ## should contains prefix sagemaker-xxx

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
# region=${region:-us-east-1} ###REGION

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} -f Dockerfile.jre .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon  180.1MB
Step 1/7 : From 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39-cu117-ubuntu20.04-sagemaker
1.13.1-gpu-py39-cu117-ubuntu20.04-sagemaker: Pulling from pytorch-training
f0412dfb1aae: Pulling fs layer
ccfccf24900f: Pulling fs layer
6eeb573e3fe0: Pulling fs layer
ba47ed544755: Pulling fs layer
e5c76c058a44: Pulling fs layer
d00bf087c1f8: Pulling fs layer
e80d6cb72455: Pulling fs layer
9e3e40817eb3: Pulling fs layer
8eeed70bef1a: Pulling fs layer
f4c09fad301f: Pulling fs layer
88f96bd0767d: Pulling fs layer
56f765e0a8c0: Pulling fs layer
e2f0ad2c6fd4: Pulling fs layer
c2bd146a72f4: Pulling fs layer
aa32a392953a: Pulling fs layer
c99f23740d75: Pulling fs layer
c6e6da6405c8: Pulling fs layer
d00bf087c1f8: Waiting
0cd9d0299ea5: Pulling fs layer
9d1f0f2412e8: Pulling fs layer
63a158a53354: Pulling fs layer
54bc2ee21cb9: Pulling fs layer
b3f13c19fde1: Pulling fs layer
e80d6cb72455: Waiting
f0d3627f0eb8:

In [ ]:
633205212955.dkr.ecr.us-east-1.amazonaws.com/sagemaker-torch113-cu117-jre:latest

In [9]:
!docker images

REPOSITORY                                                                       TAG         IMAGE ID       CREATED          SIZE
633205212955.dkr.ecr.us-east-1.amazonaws.com/sagemaker-nv-meg-iad-nvefa-0904-1   latest      7d7a3f6bb832   45 minutes ago   21GB
sagemaker-nv-meg-iad-nvefa-0904-1                                                latest      7d7a3f6bb832   45 minutes ago   21GB
<none>                                                                           <none>      1026f4677140   8 hours ago      19.9GB
nvcr.io/nvidia/pytorch                                                           23.07-py3   bd784c42fdf0   8 weeks ago      19.8GB


In [12]:
!docker rmi -f 8bb0e69ffa35

Error: No such image: 52628f47182d
